In [1]:
# TODO in the price_quality komen nu infs, vervangen door nan?

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split

import pandas as pd
import random
import csv
import os
import math
import numpy as np

from numba import jit
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.mode.chained_assignment = None

from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
# read in the dataframe you want to clean, can be a csv, hdf
traindf = pd.read_csv("./data/training_set_VU_DM.csv")

# put filename here: we use hdf because it can store the entire dataset, while pickle files can't
filename = "./data/traindf_clean.hdf"

# if testing set is True we will not add the importance column (because we can't)
testing_set = False

In [3]:
def competitors(df):
    """
    Make a new column in the dataframe (competitor_bool) for when there 
    exists a competitor and there are available rooms.
    1 is True, 0 is False.
    """

    # we say that there is no competitor with a lower price
    df["competitor_lower"] = 0 #competitor_bools
    
    # comp1rate = 1 if price is lower
    df["competitor_lower"][df["comp1_rate"] == 1] = 1
    df["competitor_lower"][df["comp2_rate"] == 1] = 1
    df["competitor_lower"][df["comp3_rate"] == 1] = 1
    df["competitor_lower"][df["comp4_rate"] == 1] = 1
    df["competitor_lower"][df["comp5_rate"] == 1] = 1
    df["competitor_lower"][df["comp6_rate"] == 1] = 1
    df["competitor_lower"][df["comp7_rate"] == 1] = 1
    df["competitor_lower"][df["comp8_rate"] == 1] = 1
    
    # we say at first there is no competitor hotel available
    df["competitor_available"] = 0
    
    # availability bool = 1 if there if the competitor and expedia are available
    df["competitor_available"][df["comp1_inv"] == 1] = 1
    df["competitor_available"][df["comp2_inv"] == 1] = 1
    df["competitor_available"][df["comp3_inv"] == 1] = 1
    df["competitor_available"][df["comp4_inv"] == 1] = 1
    df["competitor_available"][df["comp5_inv"] == 1] = 1
    df["competitor_available"][df["comp6_inv"] == 1] = 1
    df["competitor_available"][df["comp7_inv"] == 1] = 1
    df["competitor_available"][df["comp8_inv"] == 1] = 1

    return df


In [4]:
def visitor_history(df):
    """
    Add column that tells us whether someone has visited a hotel before.
    Column name = total_visited; 1 is True, 0 is False.
    """
    
    # most visitors haven't visited a hotel yet
    df["visited_before"] = 0
    
    # where there is a history field filled in, visited_before is turned into 21
    df["visited_before"][df["visitor_hist_starrating"].notna() | df["visitor_hist_adr_usd"].notna()] = 1
    
    return df

In [5]:
def price_quality(df):
    """
    Add a column of ratio price/quality to the DataFrame.
    """
    
    df["price_quality"] = None
    
    df["prop_starrating"].replace(0, 0.0001,inplace=True) # TODO
    
    df["price_quality"][df["price_usd"].notna() & df["prop_starrating"].notna() & df["prop_starrating"] != 0] = df["price_usd"] / df["prop_starrating"]

    # Replace missing values with median
    df["price_quality"].fillna((df["price_quality"].median()), inplace=True)
    
    return df 



In [6]:
def price_category(df):
    """
    Add a column of categories of price_usd and a column
    that corrected price for number of nights.
    Preprocessing of quantile cut showed that categories are:
    [(6.0889999999999995, 69.0] < (69.0, 90.0] < (90.0, 110.0] 
    < (110.0, 136.0] < (136.0, 170.077] < (170.077, 239.0] < (239.0, 554655.0]]
    
    """
    
    # Correct for number of nights ad add as new column
    df["price_correction"] = df["price_usd"] / df["srch_length_of_stay"]
    
    # Replace missing values
    df["price_correction"].fillna((df["price_correction"].median()), inplace=True)
    df["price_usd"].fillna((df["price_usd"].median()), inplace=True)
    
    # Make a new column of price categories
    # TODO: apparently this is not a number
    df['PriceBand'] = pd.qcut(df["price_correction"], 7, labels=np.arange(1,8))
    
    return df

In [7]:
def process_remaining_cols(df):
    """
    Add some remaining (and interesting columns) to the dataframe.
    """
    
    # Replace missing values with median
    df["prop_brand_bool"].fillna((df["prop_brand_bool"].median()), inplace=True)
    df["random_bool"].fillna((df["random_bool"].median()), inplace=True)
    
    # Boolians
    df["prop_location_score1"].fillna((df["prop_location_score1"].mean()), inplace=True)
    df["prop_location_score2"].fillna((df["prop_location_score2"].mean()), inplace=True)
        
     # Replace missing values with mean
    df["promotion_flag"].fillna((df["promotion_flag"].mean()), inplace=True)
    
    return df

In [8]:
def add_score(df):
    """
    Add an importance score based on click_bool and booking_bool
    """
    
    # every hotel that is clicked on gets an importance score of 1
    df["importance"] = df["click_bool"]
    
    # every hotel that is booked gets an importance score of 5 
    df["importance"][df["booking_bool"] == 1] = 5
        
    return df

In [35]:
def prop_id_score(df):
    """
    The average price and standard deviation per prop_id
    """ 
    
    df["prop_id_price_mean"] = df.groupby('prop_id')['price_quality'].transform('mean')
    df["prop_id_price_std"] = df.groupby('prop_id')['price_quality'].transform('std')

    return df

### Aparte functies aanroepen voor de kolommen die je erbij wilt

In [11]:
traindf = competitors(traindf)
print("cleaned competitors")

traindf = visitor_history(traindf)
print("cleaned visitor history")

traindf = price_quality(traindf)
print("cleaned price quality")

traindf = price_category(traindf)
print("cleaned price category")

traindf = process_remaining_cols(traindf)
print("did remaining columns")

 # Add relevant columns    
if testing_set is False:

    df = traindf[["prop_id", 
                  "srch_id", 
                  "position", 
                  "price_quality", 
                  "competitor_lower", 
                  "competitor_available", 
                  "visited_before", 
                  "click_bool", 
                  "booking_bool", 
                  "PriceBand", 
                  "price_usd"
                  "promotion_flag", 
                  "prop_brand_bool", 
                  "random_bool",
                  "prop_location_score1", 
                  "prop_location_score2",
                  "prop_id_price_mean",
                  "prop_id_price_std"]]
    
    df = add_score(df)
    print("added score")
    
    df = prop_id_score(df)
    print("added prop_id_score")

    df.drop(columns=["click_bool", "booking_bool"])
else:
    
    # df without click_bool, booking_bool and position
    df = traindf[["prop_id", 
                  "srch_id", 
                  "price_quality", 
                  "competitor_lower", 
                  "competitor_available", 
                  "visited_before", 
                  "PriceBand", 
                  "price_usd",
                  "promotion_flag", 
                  "prop_brand_bool", 
                  "random_bool",
                  "prop_location_score1", 
                  "prop_location_score2",
                  "prop_id_price_mean",
                  "prop_id_price_std"
                 ]]

cleaned competitors
cleaned visitor history
cleaned price quality
cleaned price category
did remaining columns


In [33]:
display(df.head())
display(df.describe())

added score
added prop_id_score


,prop_id,srch_id,position,price_quality,competitor_lower,competitor_available,visited_before,click_bool,booking_bool,PriceBand,price_usd,promotion_flag,prop_brand_bool,random_bool,prop_location_score1,prop_location_score2,importance,prop_id_score
0,893,1,27,34.923333,0,0,0,0,0,5,104.77,0,1,1,2.83,0.0438,0,25.240196
1,10404,1,26,42.685000,0,1,0,0,0,7,170.74,0,1,1,2.20,0.0149,0,22.723842
2,21315,1,21,59.933333,0,0,0,0,0,7,179.80,0,1,1,2.20,0.0245,0,23.588022
3,27348,1,34,301.385000,0,1,0,0,0,7,602.77,0,1,1,2.83,0.0125,0,24.063043
4,29604,1,4,35.895000,0,0,0,0,0,6,143.58,0,1,1,2.64,0.1241,0,12.846617


,prop_id,srch_id,position,price_quality,competitor_lower,competitor_available,visited_before,click_bool,booking_bool,price_usd,promotion_flag,prop_brand_bool,random_bool,prop_location_score1,prop_location_score2,importance,prop_id_score
count,4.958347e+06,4.958347e+06,4.958347e+06,4.958347e+06,4.958347e+06,4.958347e+06,4.958347e+06,4.958347e+06,4.958347e+06,4.958347e+06,4.958347e+06,4.958347e+06,4.958347e+06,4.958347e+06,4.958347e+06,4.958347e+06,4.958347e+06
mean,7.007918e+04,1.663666e+05,1.685624e+01,5.751821e+04,1.576640e-01,8.802914e-02,5.102930e-02,4.474858e-02,2.791051e-02,2.542096e+02,2.156198e-01,6.346994e-01,2.959004e-01,2.872589e+00,1.303852e-01,1.563906e-01,1.685624e+01
std,4.060992e+04,9.611223e+04,1.042566e+01,7.122742e+06,3.644257e-01,2.833373e-01,2.200575e-01,2.067514e-01,1.647165e-01,1.600124e+04,4.112517e-01,4.815144e-01,4.564465e-01,1.531011e+00,1.408432e-01,8.307484e-01,4.877066e+00
min,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
25%,3.501000e+04,8.293600e+04,8.000000e+00,2.975667e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,8.500000e+01,0.000000e+00,0.000000e+00,0.000000e+00,1.790000e+00,2.880000e-02,0.000000e+00,1.370000e+01
50%,6.963800e+04,1.665070e+05,1.600000e+01,4.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.220000e+02,0.000000e+00,1.000000e+00,0.000000e+00,2.770000e+00,1.193000e-01,0.000000e+00,1.723529e+01
75%,1.051680e+05,2.497240e+05,2.600000e+01,5.666667e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.849600e+02,0.000000e+00,1.000000e+00,1.000000e+00,4.040000e+00,1.373000e-01,0.000000e+00,2.031707e+01
max,1.408210e+05,3.327850e+05,4.000000e+01,9.443491e+09,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.972633e+07,1.000000e+00,1.000000e+00,1.000000e+00,6.980000e+00,1.000000e+00,5.000000e+00,3.800000e+01


In [19]:
# df["price_quality"] = df.price_quality.astype(np.float32)
print(df.dtypes)
print()

if df.isnull().sum().sum() != 0:    
    print("\x1b[31mMissing values: \'\x1b[0m")
    print(df.isnull().sum())
else:
    print("\x1b[31mNo missing values!! :D \'\x1b[0m")

prop_id                    int64
srch_id                    int64
position                   int64
price_quality            float64
competitor_lower           int64
competitor_available       int64
visited_before             int64
click_bool                 int64
booking_bool               int64
PriceBand               category
promotion_flag             int64
prop_brand_bool            int64
random_bool                int64
prop_location_score1     float64
prop_location_score2     float64
importance                 int64
prop_id_score            float64
dtype: object

No missing values!! :D '


## Totale dataset zonder missende waardes

In [21]:
# save the dataframe if it does not exist yet
if not os.path.exists(filename):
    df.to_hdf(filename, key="df", format="table")

In [22]:
# test if it worked
reread = pd.read_hdf(filename)

In [23]:
display(reread.describe())

,prop_id,srch_id,position,price_quality,competitor_lower,competitor_available,visited_before,click_bool,booking_bool,promotion_flag,prop_brand_bool,random_bool,prop_location_score1,prop_location_score2,importance,prop_id_score
count,4.958347e+06,4.958347e+06,4.958347e+06,4.958347e+06,4.958347e+06,4.958347e+06,4.958347e+06,4.958347e+06,4.958347e+06,4.958347e+06,4.958347e+06,4.958347e+06,4.958347e+06,4.958347e+06,4.958347e+06,4.958347e+06
mean,7.007918e+04,1.663666e+05,1.685624e+01,5.751821e+04,1.576640e-01,8.802914e-02,5.102930e-02,4.474858e-02,2.791051e-02,2.156198e-01,6.346994e-01,2.959004e-01,2.872589e+00,1.303852e-01,1.563906e-01,1.685624e+01
std,4.060992e+04,9.611223e+04,1.042566e+01,7.122742e+06,3.644257e-01,2.833373e-01,2.200575e-01,2.067514e-01,1.647165e-01,4.112517e-01,4.815144e-01,4.564465e-01,1.531011e+00,1.408432e-01,8.307484e-01,4.877066e+00
min,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
25%,3.501000e+04,8.293600e+04,8.000000e+00,2.975667e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.790000e+00,2.880000e-02,0.000000e+00,1.370000e+01
50%,6.963800e+04,1.665070e+05,1.600000e+01,4.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,2.770000e+00,1.193000e-01,0.000000e+00,1.723529e+01
75%,1.051680e+05,2.497240e+05,2.600000e+01,5.666667e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,4.040000e+00,1.373000e-01,0.000000e+00,2.031707e+01
max,1.408210e+05,3.327850e+05,4.000000e+01,9.443491e+09,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,6.980000e+00,1.000000e+00,5.000000e+00,3.800000e+01
